## Upload functions and necessary modules

In [ ]:
from bb2022_functions import *
%matplotlib inline
from Bio.SeqIO.FastaIO import SimpleFastaParser
from Bio import SeqIO
pd.options.mode.chained_assignment = None  # default='warn'

## Import and format metadata

In [ ]:
md = pd.read_csv("/Users/Diana/Documents/escuela/phd/size_fractions_copy/metadata_merged.csv")
merged = pd.read_csv("/Users/Diana/Documents/escuela/phd/size_fractions_copy/metadata_niskin.csv")
all_md = pd.read_csv("/Users/Diana/Documents/escuela/phd/size_fractions_copy/allmetadata.csv")

### Visualize metadata

In [ ]:
maxvals = plot_nutrients(all_md, 60)

## Add microbial communities

In [ ]:
#generate a dataframe from all specified amplicon
df, comm = consolidate_tables('16S')#, frac='pooled') #16S, chloroplast, or 18S
merged = merge_metadata(df, all_md)
separated, contaminants = pick_metadata(comm, merged)
newseparated = make_defract(all_md, separated)
#apply changes to taxonomy according to NCBI identified ASVs
newdf = apply_replacement(newseparated, "feature_id", "Genus") 
#newdf = apply_replacement(newdf, "feature_id", "PRSpecies") 
# or replace Genus with PRSpecies if dealing with phytoref

### Use these lines for analysis of P

In [ ]:
#this is all the weeks and depths which we selected for comparison with P
weeks_depths_with_P = newdf[newdf['size_code'] == 'P'][['weekn', 'depth']].drop_duplicates()
filtered_df = newdf.merge(weeks_depths_with_P, on=['weekn', 'depth']) #filter the original df to only keep those samples
newdf = filtered_df

#### Inspect the dna concentrations, and read depth of samples

In [ ]:
#plot of dna concentrations per sample
dnacon(newdf, depth='all', includeSL=True)

In [ ]:
#rarefaction curves per community
rarefy_curve(comm, newdf)

We can also sort the samples by their library size:

In [ ]:
newseparated[['Total','sampleid']].sort_values('Total').drop_duplicates()

In [ ]:
#separately copy newseparated from each comm into 'proks', 'euks', and 'chloro'
combined = pd.concat([proks, euks, chloro], ignore_index=True)
agg_counts = combined.groupby(['weekn','comm'])['feature_frequency'] \
                     .sum().unstack('comm')

fig, ax = plt.subplots(figsize=(12,4))
agg_counts.plot(ax=ax)
ax.set_yscale('log')
ax.set_xlabel('Week number')
ax.set_ylabel('Feature frequency (log scale)')
ax.set_title('Raw counts per community (all depths summed)')
ax.legend(title='Community', bbox_to_anchor=(1.02,1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
#this code was corrected by AI
sample_counts = newseparated.groupby('sampleid')['feature_frequency'].sum()
#calculate % kept read for various sampling depths to pick one
percentiles = [10, 25, 50, 75]
depths = [int(np.percentile(sample_counts, p)) for p in percentiles]
depths += [int(sample_counts.min()), int(sample_counts.max())]
depths = sorted(set(depths))

retained = [(sample_counts >= d).sum() for d in depths]
percent_retained = [r / len(sample_counts) * 100 for r in retained]
table = pd.DataFrame({
    'Depth': depths,
    'Samples Retained': retained,
    'Percent Retained (%)': np.round(percent_retained, 1)
})
table

In [ ]:
#Added repeat-rarefaction step for alpha diversity comparisons
rarefaction_depths = [5000, 10000]  # example depths

df_merged = newseparated.copy()
feature_table = newseparated.pivot_table(
    index='sampleid', columns='feature_id', values='feature_frequency', fill_value=0
)

for depth in rarefaction_depths:
    summary_stats = repeat_rarefy(feature_table, depth=depth, n_iter=100)
    summary_stats_flat = summary_stats.copy()
    summary_stats_flat.columns = ['_'.join(col) + f"_{depth}" for col in summary_stats_flat.columns]
    summary_stats_flat = summary_stats_flat.reset_index()
    # Merge with the cumulative df
    df_merged = df_merged.merge(summary_stats_flat, on='sampleid', how='left')

In [ ]:
sizecodes = ['S', 'L', 'W', 'SL']
palette_colors = sns.color_palette()
palette_dict = {sizecode: color for sizecode, color in zip(sizecodes, palette_colors)}

#print y=Total to check library depth, and y=nASVs
d1_rarefied = df_merged[df_merged["depth"] == 5]
sns.lineplot(data=d1_rarefied.sort_values('weekn'), x='weekn', y='richness_mean_5000', marker='o', hue='size_code',
            palette=palette_dict)
plt.tight_layout()
plt.show()

#### Explore the taxonomy in the samples and compare

In [ ]:
#Produce interactive taxonomic barplots with plotly
phyld, top10d = taxbarplot(comm, newdf, 'Genus', depth=60, topn=10, colrow='size_code')

In [ ]:
#Visualize the static barplots with seaborn, and each size fraction separately
taxonomic_barplots(comm, newdf, [5,60], 'Genus', 21, False)

In [ ]:
taxonomic_barplots_p(comm, newdf, [5,60], 'Genus', 21, True)

In [ ]:
#Generate the heatmap for the top genus from each sample
heatmap_top1(comm, newdf, 'Genus')

In [ ]:
#use this to inspect taxonomy of specific feature ids 'f_id'
f_id = '25679bd7ae54946d9d7348b7fde04db4'
newdf.loc[newdf['feature_id'] == f_id, 'Taxon'].tolist()[0]

The above plot uses taxonomy, but we can generate the same plot but by comparing whether 80% of the features in each samples are also found in the whole (unfractionated samples). This was quantified by dividing the number of shared features by the toal number of features. If a square has a red color (closer to 1), it's very similar to the unfractionated sample, and the bluer the square, the more different it is from the unfractionated.

In [ ]:
grab_80perc(comm, newseparated, 0.8, 'feature_id')

We can plot alpha diversity measurements, whether as 'shannon_diversity' or 'nASVs' which is the richness quantified by the total number of ASVs

In [ ]:
#run the visualisations for alpha diversity and run pairwise t-tests between size fractions for richness values
anova, results = boxplot_depth(newdf, comm, 60, 'nASVs', 'Shannon Diversity Index')
#results gives the corrected p-values for pairwise comparisons

In [ ]:
results

Compare the slopes of linear regressions of the richness change over time. Each value represents how much a size fraction (column) differs in comparison to the average slope (averaged between all size fractions) for each depth (rows).

In [ ]:
tohm, z_sc_df = get_slopes(comm, separated)
#a zscore of 1= 1 std away from the mean,
#positive values=higher than mean, neg= smaller than mean

### Venn diagrams for features unique to size fractions

In [ ]:
dfplot, level, dfplot_unweighted = calcperc_defrac_unweighted(comm, newdf, 'feature_id')
dfplot, level, dfplot_unweighted = calcperc_defrac(comm, newdf, 'feature_id', dfplot_unweighted)

In [ ]:
f_id = '000527117f05c819fdf7268c540a4a3b'
newdf.loc[newdf['feature_id'] == f_id, 'Taxon'].tolist()[0]

In [ ]:
timeseries_fid(comm, newseparated, f_id, 's__uncultured_Alphaproteobacteria', 30)

### Beta diversity and ANCOM analysis

Optionally we can run ANCOM with removed low abundance features with a given threshold

In [ ]:
#only if we want to run ANCOM pairwise
news2 = newseparated[newseparated.size_code != 'L']
news2 = news2[news2.size_code != 'SL']

In [ ]:
depths = [1,5,10,30,60]
for depth in depths:
    pca, pca_features, sfdclr, dm = pcaplot(newdf, depth, comm, 'size_code', 'DFr', 'week')

In [ ]:
depths = [1,5,10,30,60]
for depth in depths:
    pca, pca_features, sfdclr, dm = pcaplot(newdf, depth, comm, 'size_code', 'DFr', 'week')
    DAresults, DARejected_SC_taxonomy, prcentile = run_ancom(comm, newdf, sfdclr, depth, 'size_code', threshold=0)

    #save outputs
    DAresults.to_csv('outputs/ANCOM/chloroplast/none/'+comm+'_D'+str(depth)+'_WSLSL.csv')
    DARejected_SC_taxonomy.to_csv('outputs/ANCOM/chloroplast/none/'+comm+'_D'+str(depth)+'_Trueonly_WSLSL.csv')

    notify()

Bray-curtis analysis of SL against W pre-bloom and bloom

In [ ]:
ft = newseparated.pivot_table(
    index=['weekn', 'depth', 'size_code'],
    columns='feature_id',
    values='feature_frequency',
    fill_value=0
)

### check if clr transform from sci py
def clr_transform(arr):
    return clr(arr + 0.1)


rows = []
for (week, depth), group in ft.groupby(level=[0, 1]):
    # try to extract SL and W for this (week, depth)
    try:
        sl = group.xs('SL', level=2).values.flatten()
        w  = group.xs('W',  level=2).values.flatten()
    except KeyError:
        continue  # skip if SL or W is missing

    # clr transformation
    sl_clr = clr_transform(sl)
    w_clr  = clr_transform(w)

    # compute Bray–Curtis
    bc = braycurtis(sl_clr, w_clr)

    # determine bloom stage here
    stage = 'Pre-bloom' if week < 8 else 'Bloom'

    rows.append({
        'weekn': week,
        'depth': depth,
        'braycurtis': bc,
        'stage': stage
    })

bc_df = pd.DataFrame(rows)

print("Columns in bc_df:", bc_df.columns.tolist())

sns.catplot(
    data=bc_df,
    x='stage',
    y='braycurtis',
    hue='depth',
    kind='point',
    dodge=True,
    capsize=0.1,
    height=5,
    aspect=1 
)
plt.ylabel('Bray-Curtis distance (SL and W)')
plt.show()

Depending on ancom results, we can investigate single features temporal dynamics

In [ ]:
f_id = 'f3aa3ab8b0d2a94859675d59169af75'
newdf.loc[newdf['feature_id'] == f_id, 'Taxon'].tolist()[0]

Visualize the time series of a single feature in each size fraction over the 16 weeks

In [ ]:
timeseries_fid(comm, newseparated, f_id, 'g__Caenarcaniphilales', 1)

In [ ]:
feature_id_summary = count_feature_id_presence_with_depth_and_W('outputs', comm)
top_asvs_summary = filter_top_asvs(feature_id_summary, method="top_W_sum", n=50)
plot_asv_heatmap(comm, feature_id_summary, file_filter="WSLSL")

### Export files

#### Files for NCBI

To generate MIMARKS file for NCBI sequence submission; the output is a .csv file for the samples and their metadata for submission (i.e sampleid, size fraction, date)

In [ ]:
make_MIMARKS(newseparated)

#### Files for R

To create a phyloseq object, you need an ASV table, taxonomy file and metadata

In [ ]:
correlation_df = correlation_df.sort_values(by='correlation', ascending=False)

In [ ]:
correlation_df.sort_values('correlationé')